<a href="https://colab.research.google.com/github/pvoipt/AiDeveloping/blob/main/TickNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical  # one-hot encoding 을 위한 함수
from tensorflow.keras.models import Sequential  # 레이어를 층층히 쌓아가는 연쇄 모델
from tensorflow.keras.layers import Dense  # 완전연결층
from tensorflow.keras.models import load_model  # 저장된 모델 불러오기

"""퍼셉트론 설계
      데이터 처리 용이함을 위해 행렬 사용

   신경망 설계
      각 퍼셉트론에서 다음 퍼셉트론으로 값을 전달하는 것을 시간 1로 침(모든 퍼셉트론이 동시에 작동하게 하기 위함)
      작동 방법
        1. 지정 숫자만큼 퍼셉트론 생성
        2. 퍼셉트론별로 

   퍼셉트론 클래스, 신경망 생성 클래스를 따로
"""

In [16]:
def stepF(X):
  if np.sum(X) > 0.3:
    return 1
  else:
    return 0

In [17]:
def ReLU(X):
  return np.maximum(0, np.sum(X))

In [18]:
def SSE(y, t):    #sum of squares for error, 오차제곱합
  return 0.5 * np.sum((y - t)**2)

In [19]:
def randGene(n):      #무작위 유전자 생성
  return np.random.binomial(1, 0.5, n)

In [20]:
def GTV(gene):  #Gene To Value
  if isinstance(gene, np.float64):
    return 1
  else:
    bi = ''
    for i in gene:
      bi += str(i)
    return int(bi, 2)

In [21]:
class Perceptron:
  def __init__(self, size, nSpN, maxInput, role, sumType, nextSize):
    self.role = role
    #if role == "sensory":       #감각뉴런이라면 가중치 1 (원본 그대로 반영)
      #self.wgtGene = np.ones(100)
    if role == "inter":       #연결뉴런이라면 가중치 0~1 랜덤
      self.wgtGene = []
      for i in range(maxInput):
        self.wgtGene.append(randGene(8))
    elif role == "motor":
      pass
    self.size = size
    if nextSize == -1:
      self.nextSize = size
    else:
      self.nextSize = nextSize
    self.sumType = sumType
    self.nSpN = nSpN    # nSpN = the number of Synapse per Neuron : 뉴런별로 뻗어나갈 시냅스 수
    self.X = []       #입력값 받는 리스트
    self.summation = 0      #가중치을 더하는 변수 (시간적 가중 용)
    self.maxInput = maxInput #최대 입력(최대 연결될 수 있는 시냅스 수)(넘어가면 무시)
  def connectNeuron(self):
    self.cN = np.random.randint(0, self.nextSize, self.nSpN)    #랜덤하게 시냅스 연결
  def calWgt(self):       #가중치 유전자를 값으로 변환
    self.wgt  = []
    for i in range(self.maxInput):
      self.wgt = np.append(self.wgt, GTV(self.wgtGene[i])/ 2**8)
  def input_(self, X):    #값 입력받음
    if len(self.X) + len(X) < self.maxInput or self.role == "motor":
      self.X += X
  def out(self):
    if self.role == "sensory":
      self.WX = self.X
    if self.role == "inter":
      self.WX = np.array(self.X) * self.wgt[:len(self.X)]   #가중치와 입력 곱함
      self.X = []
    if self.role == "motor":
      return 0
    else:
      if self.sumType == "spatial":      #공간적/시간적 가중에 따라 활성화 방식 다름
        return stepF(self.WX)
      elif self.sumType == "temporal":
        self.summation += np.sum(self.WX)
        if self.summation >= 1:
          self.summation -= 1
          return 1
        else:
          return 0

In [22]:
class Network:
  def __init__(self, size, nSpN, maxInput, nInputLayer, nOutputLayer): #nSpN = the number of Synapse per Neuron
    self.size = size
    self.nSpN = nSpN
    self.maxInput = maxInput
    self.inputLayer = []
    self.neuron = []
    self.nInputLayer = nInputLayer
    self.nOutputLayer = nOutputLayer
    self.inputTable = []
  def makeup(self):
    for i in range(self.nInputLayer):         #입력뉴런(감각뉴런) 생성
      n = Perceptron(self.size, self.nSpN, self.maxInput, "sensory", "temporal", self.size)
      n.connectNeuron()
      self.inputLayer.append(n)
    for i in range(self.size):      #연결뉴런 생성
      n = Perceptron(self.size, self.nSpN, self.maxInput, "inter", "temporal", self.size + self.nOutputLayer)
      n.connectNeuron()
      self.neuron.append(n)
    for i in range(self.nOutputLayer):
      n = Perceptron(self.size, self.nSpN, self.maxInput, "motor", "temporal", -1)
      self.neuron.append(n)
  def calNeuronWgt(self):
    for i in range(self.size):
      self.neuron[i].calWgt()
  def input_(self):     #입력층 입력
    for i in range(self.size + self.nOutputLayer):
      self.inputTable.append([])
    for i in range(self.nInputLayer):
      r = self.inputLayer[i].out()
      for j in range(self.nSpN):
        self.inputTable[self.inputLayer[i].cN[j]].append(r)
  def tick(self):       #시간 경과 (틱)
    for i in range(self.size):
      r = self.neuron[i].out()
      for j in range(self.nSpN):
        self.inputTable[self.neuron[i].cN[j]].append(r)
        
    for i in range(self.size + self.nOutputLayer):
      if len(self.inputTable[i]):
        self.neuron[i].input_(self.inputTable[i])
    self.inputTable = []
    for i in range(self.size + self.nOutputLayer):
      self.inputTable.append([])
  def output(self):     #출력층 출력
      self.out = []
      for i in range(self.size, self.size + self.nOutputLayer):
        self.out.append(np.sum(self.neuron[i].X))
      return np.array(self.out) / np.sum(self.out)

In [23]:
def breed(p1, p2):        #자손 생성 함수 p1 : 부모 1, p2 : 부모 2
  offspring = p1
  #newGene = []
  newChromosome = []
  for i in range(p1.size):
    for j in range(p1.maxInput):
      rand = np.random.binomial(1, 0.5, 1)[0]    #50% 확률 구현
      if rand:
        newChromosome.append(p1.neuron[i].wgt[j])
      else:
        newChromosome.append(p2.neuron[i].wgt[j])
    #newGene.append(newChromosome)

  for i in range(offspring.size):
    offspring.neuron[i].wgt = newChromosome
  
  return offspring
  

In [24]:
#MNIST
(X_train, y_train), (X_test, y_test) = mnist.load_data()

input_shape = X_train.shape[1] * X_train.shape[2] #그림 크기 (28*28)
number_of_classes = len(set(y_train))

X_train = X_train / 255.0  #정규화
X_test = X_test / 255.0
X_train = X_train.reshape(-1, input_shape) #차원 낮추기
X_test = X_test.reshape(-1, input_shape)

y_train = to_categorical(y_train, number_of_classes) #원-핫 인코딩
y_test = to_categorical(y_test, number_of_classes)

In [25]:
print(np.shape(X_train))

X_train[0][550]

(60000, 784)


0.9803921568627451

In [26]:
def test(n):
  total = []
  for a in range(10):
    t = 0
    for i in range(10):
      for j in range(784):
        n[a].inputLayer[j].input_([X_train[i][j]])
      for k in range(10):
        n[a].input_()
        n[a].tick()
      t += SSE(y_train[i], n[a].output())
    total.append(t)
  return total

In [27]:
def shift_of_generation(n):
  total = []
  total = test(n)
  print(total)
  p1 = np.argmin(total)
  del total[p1]
  p2 = np.argmin(total)
  o = []
  for i in range(10):             #자손 10 생성
    o.append(breed(n[p1], n[p2]))   
  return o  


In [28]:
n = []
for i in range(40):
  n.append(Network(10000, 10, 30, 784, 10))
  n[i].makeup()
  n[i].calNeuronWgt()

gen5 = shift_of_generation(shift_of_generation(shift_of_generation(shift_of_generation(shift_of_generation(shift_of_generation(n))))))

[4.563142440158856, 4.54912801353888, 4.625440488036966, 4.512960270897025, 4.488392855192647, 4.50179929842273, 4.682518583202333, 4.4700211587137515, 4.42750494813523, 4.482326532756777]
[4.432373920165144, 4.432735325647845, 4.432892128842143, 4.432980397288262, 4.433037112894398, 4.433076657427046, 4.4331058134607435, 4.433128203358338, 4.433145939366873, 4.433160336819053]
[4.433172257781174, 4.433182290905543, 4.433190851906598, 4.4331982427462755, 4.433204688062503, 4.433210358437351, 4.433215385737934, 4.433219873512352, 4.433217794870728, 4.433220970515146]
[4.433221073455684, 4.433220648955638, 4.433220567728069, 4.433220221413967, 4.433220226915621, 4.433219935516965, 4.433220310532299, 4.4332202975573125, 4.433220049311805, 4.433220223265112]
[4.4332173595650595, 4.433217235347425, 4.433217407842543, 4.433217646411563, 4.433217532026029, 4.433217570560408, 4.4332177899094765, 4.433217686011388, 4.433217764775271, 4.433217669358889]
[4.433217771854627, 4.433217682901215, 4.4